In [ ]:
import numpy as np
import math
import re
import pandas as pd
from bs4 import BeautifulSoup

from google.colab import drive

In [ ]:
try:
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow.keras import layers
import tensorflow_datasets as tfds

In [ ]:
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
cols = ['sentiment', 'id', 'date','query', 'user', 'text']
train_data = pd.read_csv(
    '/content/drive/MyDrive/trainingandtestdata/training.1600000.processed.noemoticon.csv',
    header=None, 
    names=cols, 
    engine='python', 
    encoding='latin1')

test_data = pd.read_csv(
    '/content/drive/MyDrive/trainingandtestdata/testdata.manual.2009.06.14.csv',
    header=None, 
    names=cols, 
    engine='python', 
    encoding='latin1')

In [ ]:
data = train_data

In [ ]:
data.head()

,sentiment,id,date,query,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
data.drop(['id', 'query', 'date', 'user'], axis=1, inplace=True)


In [ ]:
def clean_tweet(tweet):
  tweet = BeautifulSoup(tweet, 'lxml').get_text()
  tweet = re.sub(r"@[A-Za-z0-9]+", " ", tweet)
  tweet = re.sub(r"https?://[A-Za-z0-9./]+", " ", tweet)
  tweet = re.sub(r" +", " ", tweet)
  return tweet

In [ ]:
data_clean = [clean_tweet(tweet) for tweet in data.text]

In [ ]:
data_labels = data.sentiment.values
data_labels[data_labels == 4] = 1

In [ ]:
set(data_labels)

{0, 1}

In [ ]:
tokenizer = tfds.features.text.SubwordTextEncoder.build_from_corpus(
    data_clean, target_vocab_size=2**16
)
data_inputs = [tokenizer.encode(sentence) for sentence in data_clean]

AttributeError: ignored

In [ ]:
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    data_clean, target_vocab_size=2**16
)
data_inputs = [tokenizer.encode(sentence) for sentence in data_clean]

In [ ]:
MAX_LEN = max([len(sentence) for sentence in data_inputs])
data_inputs = tf.keras.preprocessing.sequence.pad_sequences(data_inputs, value=0, padding='post', maxlen=MAX_LEN)

In [ ]:
test_idx = np.random.randint(0,800000,8000)
test_idx = np.concatenate((test_idx, test_idx + 800000))

In [ ]:
test_inputs = data_inputs[test_idx]
test_labels = data_labels[test_idx]
train_inputs = np.delete(data_inputs, test_idx, axis=0)
train_labels = np.delete(data_labels, test_idx)

In [ ]:
class DCNN(tf.keras.Model):

  def __init__(self, vocab_size, emb_dim=128, 
               nb_filters=50, FFN_units=512, 
               nb_classes=2, dropout_rate=0.1, 
               training=False, name='dcnn'):
    super(DCNN, self).__init__(name=name)

    self.embedding = layers.Embedding(vocab_size, emb_dim)
    self.bigram = layers.Conv1D(filters=nb_filters, kernel_size=2, padding='valid', activation='relu')
    self.pool_1 = layers.GlobalMaxPooling1D()
    self.trigram = layers.Conv1D(filters=nb_filters, kernel_size=3, padding='valid', activation='relu')
    self.pool_2 = layers.GlobalMaxPooling1D()
    self.fourgram = layers.Conv1D(filters=nb_filters, kernel_size=4, padding='valid', activation='relu')
    self.pool_3 = layers.GlobalMaxPooling1D()
    self.dense_1 = layers.Dense(units=FFN_units, activation='relu')
    self.dropout = layers.Dropout(rate=dropout_rate)
    if nb_classes==2:
      self.last_dense = layers.Dense(units=1, activation='sigmoid')
    else:
      self.last_dense = layers.Dense(units=nb_classes, activation='softmax')
  
  def call(self, impute, training):
    x = self.embedding(impute)
    x_1 = self.bigram(x)
    x_1 = self.pool_1(x_1)
    x_2 = self.trigram(x)
    x_2 = self.pool_1(x_2)
    x_3 = self.fourgram(x)
    x_3 = self.pool_1(x_3)

    merged = tf.concat([x_1, x_2, x_3], axis = 1)  #batchsize, 3  * nb_filters
    merged = self.dense_1(merged)
    merged = self.dropout(merged, training)
    output = self.last_dense(merged)

    return output


In [ ]:
VOCAB_SIZE = tokenizer.vocab_size

EMB_DIM = 200
NB_FILTERS = 100
FFN_UNITS = 256
NB_CLASSES = len(set(train_labels))

DROPOUT_RATE = 0.2

BATCH_SIZE = 32
NB_EPOCHS = 5

In [ ]:
Dcnn = DCNN(vocab_size=VOCAB_SIZE,
            emb_dim=EMB_DIM,
            nb_filters=NB_FILTERS,
            FFN_units=FFN_UNITS,
            nb_classes=NB_CLASSES,
            dropout_rate=DROPOUT_RATE)

In [ ]:
if NB_CLASSES == 2:
  Dcnn.compile(loss='binary_crossentropy',
               optimizer='adam',
               metrics=['accuracy'])
else:
  Dcnn.compile(loss='sparse_categorical_crossentropy',
               optimizer='adam',
               metrics=['sparse_categorical_crossentropy'])

In [ ]:
checkpoint_path = "./drive/MyDrive/trainingandtestdata/ckpt/"

ckpt = tf.train.Checkpoint(Dcnn=Dcnn)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=1)

if ckpt_manager.latest_checkpoint:
  ckpt.restore(ckpt_manager.latest_checkpoint)
  print("Latest checkpoint restored")

In [ ]:
Dcnn.fit(train_inputs,
         train_labels,
         batch_size=BATCH_SIZE,
         epochs=NB_EPOCHS)
ckpt_manager.save()

Epoch 1/5
49503/49503 [==============================] - 19009s 384ms/step - loss: 0.4229 - accuracy: 0.8042
Epoch 2/5
   89/49503 [..............................] - ETA: 5:16:16 - loss: 0.2841 - accuracy: 0.8794

KeyboardInterrupt: ignored

In [ ]:
results = Dcnn.evaluate(test_inputs, test_labels, batch_size = BATCH_SIZE)
print(results)

In [ ]:
Dcnn(np.array([tokenizer.encode("Your sentence")]), training=False.numpy())